In [ ]:
from flask import Flask, jsonify, request
import psycopg2
from sqlalchemy import create_engine, Column, Integer, DOUBLE_PRECISION, TIMESTAMP
from sqlalchemy.orm import sessionmaker, declarative_base
import pandas as pd
import os

app = Flask(__name__)

# Database configuratio
DB_CONFIG = {
    "host": os.environ['DBHOST'],
    "port": os.environ['DBPORT'],
    "dbname": os.environ['DBNAME'],
    "user": os.environ['DBUSER'],
    "password": os.environ['DBPASSWORD']
}

# Define PostgreSQL string
postgresql_url = "postgresql://" + DB_CONFIG["user"] + ":" + DB_CONFIG["password"] + "@" + DB_CONFIG["host"] + ":" + DB_CONFIG["port"] + "/" + DB_CONFIG["dbname"]

# Create an engin
engine = create_engine(postgresql_url)

# Configure a Session class and bind to engine
Session = sessionmaker(bind=engine)

# Define SQLAlchemy model
Base = declarative_base()

# Define sensor data class
class Sensor_Data(Base):
    __tablename__ = 'sensor_data'
    id = Column(Integer, primary_key=True)
    temperature = Column(DOUBLE_PRECISION)
    humidity = Column(DOUBLE_PRECISION)
    co2 = Column(DOUBLE_PRECISION)
    light = Column(DOUBLE_PRECISION)
    soil_moisture = Column(DOUBLE_PRECISION)
    timestamp = Column(TIMESTAMP)

# Fetch sensor data (GET request)
@app.route('/fetch-sensor-data', methods=['GET'])
def fetch_sensor_data():
    try:
        # Create a session
        # session = Session()

        # Query database
        # sensor_data = session.query(Sensor_Data).all()

        # Close the session
        # session.close()

        # Convert to a list of dictionaries
        # dicts = sensor_data.to_dict(orient="records")

        # Instantiate engine
        conn = engine.connect()

        # Retrieve data
        dataFrame = pd.read_sql("Select * from \"sensor_data\"", conn)

        # Close engine
        conn.close()

        # Convert to dict
        dicts = dataFrame.to_dict(orient="records")
        
        # Return the sensor data as a JSON response
        return jsonify(dicts), 200
    except Exception as e:
        # Error handling
        return jsonify({"error": f"Error retrieving sensor data: {str(e)}"}), 500


# Post sensor data (POST request)
@app.route('/fetch-sensor-data', methods=['POST'])
def post_sensor_data():
    try:
        # Receive sensor data from the .NET API
        sensor_data = request.get_json()

        # Validate the data (you can customize this to match your expected format)
        if not isinstance(sensor_data, list):
            return jsonify({"error": "Data should be a list of sensor readings"}), 400

        # Here, you can add logic to save the data to your database, log it, etc.
        # Example of inserting into the database (assuming the columns are temperature, humidity, etc.)
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        
        for sensor in sensor_data:
            query = """
                INSERT INTO sensor_data (temperature, humidity, co2, light, soil_moisture)
                VALUES (%s, %s, %s, %s, %s);
            """
            cursor.execute(query, (
                sensor.get('temperature', 0),
                sensor.get('humidity', 0),
                sensor.get('co2', 0),
                sensor.get('light', 0),
                sensor.get('soilMoisture', 0)
            ))

        # Commit the changes and close the connection
        conn.commit()
        conn.close()

        return jsonify({"status": "Data successfully received"}), 201
    except Exception as e:
        # Error handling
        return jsonify({"error": f"Error posting sensor data: {str(e)}"}), 500

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.96.0.73:5000
Press CTRL+C to quit
127.0.0.1 - - [07/Apr/2025 08:44:43] "GET /fetch-sensor-data HTTP/1.1" 200 -
127.0.0.1 - - [07/Apr/2025 08:44:43] "GET /favicon.ico HTTP/1.1" 404 -
